<a href="https://colab.research.google.com/github/eyadrid/Language-Translation/blob/main/Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!wget -P ./attention_keras/data https://github.com/udacity/deep-learning/raw/master/language-translation/data/small_vocab_en
!wget -P ./attention_keras/data https://github.com/udacity/deep-learning/raw/master/language-translation/data/small_vocab_fr

--2024-05-03 18:14:24--  https://github.com/udacity/deep-learning/raw/master/language-translation/data/small_vocab_en
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/udacity/deep-learning/master/language-translation/data/small_vocab_en [following]
--2024-05-03 18:14:24--  https://raw.githubusercontent.com/udacity/deep-learning/master/language-translation/data/small_vocab_en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9085267 (8.7M) [text/plain]
Saving to: ‘./attention_keras/data/small_vocab_en’

small_vocab_en      100%[===================>]   8.66M  --.-KB/s    in 0.03s   

2024-05-03 1

In [3]:
import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [4]:
english_path='https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_en.txt'
french_path='https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_fr.txt'

In [5]:
import os

def load_data(path):
  input_file = os.path.join(path)
  with open(input_file, "r") as f:
    data = f.read()

  return data.split('\n')

In [6]:
import tensorflow as tf
english_data=tf.keras.utils.get_file('file1',english_path)
french_data=tf.keras.utils.get_file('file2',french_path)

10135741/10135741 [==============================] - 0s 0us/step


In [7]:
english_sentences=load_data(english_data)
french_sentences=load_data(french_data)

In [8]:
for i in range(5):
  print('Sample :',i)
  print(english_sentences[i])
  print(french_sentences[i])
  print('-'*50)


Sample : 0
new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--------------------------------------------------
Sample : 1
the united states is usually chilly during july , and it is usually freezing in november .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
--------------------------------------------------
Sample : 2
california is usually quiet during march , and it is usually hot in june .
california est généralement calme en mars , et il est généralement chaud en juin .
--------------------------------------------------
Sample : 3
the united states is sometimes mild during june , and it is cold in september .
les états-unis est parfois légère en juin , et il fait froid en septembre .
--------------------------------------------------
Sample : 4
your least liked fruit is the grape , but my least liked is the apple .
votre moins aimé frui

In [9]:
import collections

In [10]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

In [11]:
print('English Vocab:',len(english_words_counter))
print('French Vocab:',len(french_words_counter))

English Vocab: 227
French Vocab: 355


In [12]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

In [13]:
# Tokenize Sample output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
  print('Sequence {} in x'.format(sample_i + 1))
  print('  Input:  {}'.format(sent))
  print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [14]:
def pad(x, length=None):
  return pad_sequences(x, maxlen=length, padding='post')

In [15]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    #Expanding dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [16]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''

  #So basically we are predicting output for a given word and then selecting best answer
  #Then selecting that label we reverse-enumerate the word from id
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [17]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005

    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [18]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [19]:
simple_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [20]:
simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 256)           51200     
                                                                 
 gru (GRU)                   (None, 21, 256)           394752    
                                                                 
 time_distributed (TimeDist  (None, 21, 1024)          263168    
 ributed)                                                        
                                                                 
 dropout (Dropout)           (None, 21, 1024)          0         
                                                                 
 time_distributed_1 (TimeDi  (None, 21, 345)           353625    
 stributed)                                                      
                                                                 
Total params: 1062745 (4.05 MB)
Trainable params: 106274

In [21]:
history=simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Epoch 1/10
108/108 [==============================] - 15s 73ms/step - loss: 1.3356 - accuracy: 0.6866 - val_loss: 0.4623 - val_accuracy: 0.8508
Epoch 2/10
108/108 [==============================] - 6s 57ms/step - loss: 0.3947 - accuracy: 0.8710 - val_loss: 0.2943 - val_accuracy: 0.9013
Epoch 3/10
108/108 [==============================] - 6s 59ms/step - loss: 0.2840 - accuracy: 0.9050 - val_loss: 0.2373 - val_accuracy: 0.9196
Epoch 4/10
108/108 [==============================] - 6s 59ms/step - loss: 0.2405 - accuracy: 0.9188 - val_loss: 0.2165 - val_accuracy: 0.9260
Epoch 5/10
108/108 [==============================] - 6s 58ms/step - loss: 0.2164 - accuracy: 0.9258 - val_loss: 0.2018 - val_accuracy: 0.9297
Epoch 6/10
108/108 [==============================] - 6s 57ms/step - loss: 0.2025 - accuracy: 0.9297 - val_loss: 0.1943 - val_accuracy: 0.9325
Epoch 7/10
108/108 [==============================] - 6s 58ms/step - loss: 0.1927 - accuracy: 0.9325 - val_loss: 0.1864 - val_accuracy: 0.935

In [22]:
history=simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Epoch 1/20
108/108 [==============================] - 6s 59ms/step - loss: 0.1760 - accuracy: 0.9369 - val_loss: 0.1827 - val_accuracy: 0.9367
Epoch 2/20
108/108 [==============================] - 6s 59ms/step - loss: 0.1729 - accuracy: 0.9376 - val_loss: 0.1785 - val_accuracy: 0.9372
Epoch 3/20
108/108 [==============================] - 6s 59ms/step - loss: 0.1704 - accuracy: 0.9382 - val_loss: 0.1773 - val_accuracy: 0.9378
Epoch 4/20
108/108 [==============================] - 6s 59ms/step - loss: 0.1682 - accuracy: 0.9390 - val_loss: 0.1779 - val_accuracy: 0.9380
Epoch 5/20
108/108 [==============================] - 6s 57ms/step - loss: 0.1717 - accuracy: 0.9381 - val_loss: 0.1823 - val_accuracy: 0.9367
Epoch 6/20
108/108 [==============================] - 6s 58ms/step - loss: 0.1706 - accuracy: 0.9382 - val_loss: 0.1785 - val_accuracy: 0.9379
Epoch 7/20
108/108 [==============================] - 6s 59ms/step - loss: 0.1701 - accuracy: 0.9386 - val_loss: 0.1807 - val_accuracy: 0.9370

In [24]:
simple_rnn_model.save("simple_rnn_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
def final_predictions(text):
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  y_id_to_word[0] = ''

  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2], padding='post')

  print(sentence.shape)
  print(logits_to_text(simple_rnn_model.predict(sentence[:1])[0], french_tokenizer))

In [26]:
import re

In [66]:
txt=input().lower()
final_predictions(re.sub(r'[^\w]', ' ', txt))

california is never warm during march , but it is sometimes relaxing in may .
(1, 21)
1/1 [==============================] - 0s 19ms/step
californie est jamais chaud en mois de mars mais il est relaxant relaxant en mai      
